In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv("/content/drive/MyDrive/Datasets/spam (1).csv")

In [3]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.shape

(5572, 2)

In [8]:
df.duplicated().sum()

415

In [21]:
df.isna().sum()

,0
Category,0
Message,0


In [9]:
df.drop_duplicates(inplace=True)

In [ ]:
sentence="Hey there! Did you know that machine learning (ML) and data_ science are transforming industries worldwide? Companies like Google, Amazon, and Netflix are using advanced Al models. Wanna learn more?"

In [ ]:
#removal of punctuation
import string
punc=string.punctuation
sentence1=sentence.lower()
sentence2=[i for i in sentence1 if i not in punc]
sentence3="".join(sentence2)

In [10]:
import string
punc=string.punctuation
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
stop_word=stopwords.words("english")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
from nltk.tokenize import word_tokenize
nltk.download("punkt_tab")


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [12]:
#lemmatization
from nltk.stem import WordNetLemmatizer
nltk.download("wordnet")
lemmatizer=WordNetLemmatizer()



[nltk_data] Downloading package wordnet to /root/nltk_data...


In [13]:
lemmatizer.lemmatize("bought","v")

'buy'

In [14]:
#fuction creation
def preprocess(sentence):
  sentence1=sentence.lower()
  sentence2=[i for i in sentence1 if i not in punc]
  sentence3="".join(sentence2)
  tokens=word_tokenize(sentence3)
  sentence4=[i for i in tokens if i not in stop_word]
  sentence_lem_lst=[lemmatizer.lemmatize(i,"v") for i in sentence4 ]
  return " ".join(sentence_lem_lst)

In [16]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [17]:
df_new=df.copy()

In [18]:
df["Message"]=df["Message"].apply(func=preprocess)

In [19]:
df

,Category,Message
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joke wif u oni
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor u c already say
4,ham,nah dont think go usf live around though
...,...,...
5567,spam,2nd time try 2 contact u u £750 pound prize 2 ...
5568,ham,ü b go esplanade fr home
5569,ham,pity mood soany suggestions
5570,ham,guy bitch act like id interest buy something e...


In [20]:
df["Category"].value_counts()

,count
Category,
ham,4516
spam,641


In [22]:
#COUNT VECTORIZER
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()
x_vector=vectorizer.fit_transform(df["Message"])

In [23]:
#convert
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df["Category"]=le.fit_transform(df["Category"])

In [24]:
y=df["Category"]

In [25]:
y.value_counts()

,count
Category,
0,4516
1,641


In [27]:
y.shape

(5157,)

In [26]:
x_vector.shape

(5157, 8482)

In [28]:
#balancing data
from imblearn. over_sampling import RandomOverSampler,SMOTE

In [29]:
sampler=RandomOverSampler()
x_resample,y_resample=sampler.fit_resample(x_vector,y)

In [31]:
#splitting data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_resample,y_resample,test_size=0.3,random_state=42)

In [32]:
y_resample.value_counts()

,count
Category,
0,4516
1,4516


In [33]:
#naive bayes algorithm
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(x_train,y_train)

MultinomialNB()

In [34]:
y_pred=model.predict(x_test)
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [35]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98      1369
           1       0.97      0.98      0.98      1341

    accuracy                           0.98      2710
   macro avg       0.98      0.98      0.98      2710
weighted avg       0.98      0.98      0.98      2710



In [36]:
print(accuracy_score(y_test,y_pred))

0.9771217712177122


In [37]:
print(confusion_matrix(y_test,y_pred))

[[1331   38]
 [  24 1317]]


In [39]:
#cross validation
from sklearn.model_selection import cross_val_score
print(cross_val_score(MultinomialNB(),x_vector,y,cv=6).mean())

0.9732402469068955


In [40]:
sample="Amazon is sending you a refunding of $32.64. Please reply with your bank account and routing number to receive your refund."

In [41]:
sample1=preprocess(sample)

In [42]:
sample1

'amazon send refund 3264 please reply bank account rout number receive refund'

In [43]:
test_vector=vectorizer.transform([sample1])

In [44]:
model.predict(test_vector)

array([1])

In [45]:
le.inverse_transform([0,1])

array(['ham', 'spam'], dtype=object)

In [47]:
def predict(sentence):
  var1=preprocess(sentence)
  vector=vectorizer.transform([var1])
  pred=model.predict(vector)[0]
  print(pred)
  if pred==0:
    print('Ham')
  else:
    print('Spam')

In [48]:
predict(sample)

1
Spam


In [49]:
print(model)

MultinomialNB()


In [50]:
import pickle

# Save the  model
with open("Naive_Bayes_model.pkl", "wb") as model_file:
    pickle.dump(model, model_file)

# Save the CountVectorizer
with open("Vectorizer.pkl", "wb") as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)
